In [ ]:
begin
    using Pkg
    PkgList = ["CSV",           #read .csv
                "DataFrames",   #nice framework for tables -- ~ pandas in Python
                "Ipopt",        #solver for optimization problem
                "JuMP",         #framework for optimization problem formulation -- ~ Pyomo in Python
                "Distributions" # for Chi^2
        ]       
    for package in PkgList
        if Base.find_package(package) == nothing
            Pkg.add(package)
        end
    end
    using CSV, DataFrames, Ipopt, JuMP, Distributions, Plots, LaTeXStrings
    include("src/utils.jl")
end

# Introduction:
*  **Topics covered:** State Estimation, $\chi^2$ test, optimization with JuMP, work with data (DataFrames)
* **Important notice** the results of this computer lab can be used in future HWs

# State Estimation

Example 4.1 from the book *Electric Energy Systems Analysis and Operation, A. Gomez-Exposito, A. Conejo, and C Canizares, CRS Press, 2009*

![](Network.PNG)


## A quick recap 
> 1. One should minimize, keeping in mind the Power Flow Equations between $V, \delta, P, Q$
$$
J(V,  P, Q, Pl, Ql) = \sum_{k \in \texttt{Buses}} w^V_k (V_k - \tilde{V_k})^2  + \sum_{k \in \texttt{Buses}} w^P_k (P_k - \tilde{P_k})^2 + \sum_{k \in \texttt{Buses}} w^Q_k (Q_k - \tilde{Q_k})^2 + \sum_{(k,l) \in \texttt{Lines}} w^{P}_{kl} (Pl_{kl} - \tilde{P_{kl}})^2 + \sum_{(k,l) \in \texttt{Lines}} w^{Ql}_{kl} (Q_{kl} - \tilde{Q_{kl}})^2,
$$
where $w^V, w^{P}, w^{Q}, w^{Pl}, w^{Ql}$ are the standard deviations of the measurements
> 2. To get the most likely state ($V,  P, Q, Pl, Ql$) of the system
> 3. The value of $J(V^*,  P^*, Q^*, Pl^*, Ql^*)$ can be further used to determine if the measurements are bad using $\chi^2$ test

# Problem data

## Branch data

In [ ]:
# Reading branch data 
begin
    
end


## Measurement data

In [ ]:
begin
    
end

## Divide data by types

We will to that for further convenience

In [ ]:
# Active power flow in lines


# Active power at buses

# Reactive power flow in lines

#Reactive power at buses

# Voltage at buses


In [ ]:
#give it a try


In [ ]:
#how can iterate over measured values from this table?

# State Estimation Procedure
Since the disperancy between data and out model is (in general) represented as follows,
$$
J(x) = \sum_{k=1}^n w_k (h_k(x) - y_k)^2,
$$
we need to specify $h_k$. In our case these are Power Flow Equations

## Power Flow equations
$$
    \begin{aligned}
        P_i &= V_i \sum_{j=1}^n V_j \left( G_{ij} \cos(\theta_{ij}) + B_{ij} \sin(\theta_{ij}) \right) \\
        Q_i &= V_i \sum_{j=1}^n V_j \left( G_{ij} \sin(\theta_{ij}) - B_{ij} \cos(\theta_{ij}) \right)
    \end{aligned}
$$
## Line Flows
$$
    \begin{aligned}
        P_{ij} &= V_i V_j \left( G_{ij} \cos(\theta_{ij}) + B_{ij} \sin(\theta_{ij}) \right) - G_{ij} V^2_i \\
        Q_{ij} &= V_i V_j \left( G_{ij} \sin(\theta_{ij}) - B_{ij} \cos(\theta_{ij}) \right) + (B_{ij} - b_{s,ij}) V^2_i
    \end{aligned}
$$

In [ ]:
function StateEstimation(Buses, GL, BL, Vi_M, Pi_M, Qi_M, PL_M, QL_M)
    """This function creates and solves a JuMP model, i.e., performes state estimation for a power system
        Measurements (Vi_M, Pi_M, etc.) must be presented as DataFrame and must containt 
            index columns (1 or 2, depends if it is for line or bus)
            measurements columns: "Value" -- measurements value
                                  "Weight" -- 1/σ of this measurement
            
        Arguments:
            Buses (UnitRange{Int64}): set of indices for buses
            GL    (Matrix{Float64}) : real part of admittance matrix of the grid
            BL    (Matrix{Float64}) : real part of admittance matrix of the grid
            Vi_M  (DataFrame)       : DataFrame that containts measuremets and its inverse stds for Voltage
            Pi_M  (DataFrame)       : DataFrame that containts measuremets and its inverse stds for Active Power
            Qi_M  (DataFrame)       : DataFrame that containts measuremets and its inverse stds for Reactive Power
            PL_M  (DataFrame)       : DataFrame that containts measuremets and its inverse stds for Active Power Line Flow
            QL_M  (DataFrame)       : DataFrame that containts measuremets and its inverse stds for Reactive Power Line Flow
    
        Returns:
            StateEst (JuMP.Model)   : optimized model of state estimation minimization problem
    """
    # Creating JuMP model
    
    # Define model variables
   
    # Define constraints
    
    # Define objective function
    
    # Solve the optimization and keep track of progress
    
    
    return 
end

In [ ]:
# Get the result

# Visualising iterations

In [ ]:

Objectives, _, ContstraintViolations, _, _ = get_iterations("out/out.txt")


# $\chi^2$ test

In [ ]:
parseRes(X) = round.(collect(JuMP.value.(res[Symbol(X)])), digits=3)

In [ ]:
# number of measurement
n = size(Measurements)[1] - (2 * length(Buses) - 1)
# define χ^2
d = Chisq(n)
# define several significance levels
sign_lvls = [0.1, 0.05, 0.01]
cdf_sign = [quantile(d, 1-sl) for sl in sign_lvls]
#plot cdf
xs = collect(0:0.001:25)
cdfs = [cdf(d, x) for x in xs]
plot(xs, cdfs, label="CDF")
for i in 1:length(cdf_sign)
    vline!([cdf_sign[i]], linewidth=2, linestyle=:dashdot, label=string(sign_lvls[i]) * " significance <=> " * string(1-sign_lvls[i]) * "confidence")
end
vline!([parseRes("J")], linewidth=2, linestyle=:dash, label="Our objective value")
plot!(xlabel="x")
plot!(ylabel=L"F_{\chi^2(n)}(x)")
plot!(legend=:bottomright)

# Takeaways
> 1. We know have an example of JuMP usage. **You can use that in the next homework**
> 2. We have a visualisation of optimization process
> 3. We have a visualisation of $\chi^2$ test
> 4. **Open this link and rate this class please!**
https://www.menti.com/poxia4npow